Análisis de datos SAMU: Llamadas 131
===================

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import date, datetime, time, timedelta
from babel.dates import format_date, format_datetime, format_time, format_timedelta, Locale
import locale                                    # para tratar de poner espanol
#locale = Locale('es', 'CL')
#locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')  

import os
import runpy
from pyexcel_ods import get_data

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

#importo sólo algunas de mis funciones, dejo otras excluídas....

cargo algunos scripts que me suelen servir para análisis

In [4]:
#listo los archivos
scripts = []
for dirname, dirnames, filenames in os.walk('/home/egidio/Dropbox/Compucosas/programas py'):
    for filename in filenames:
        scripts.append(os.path.join(dirname, filename))

scripts

# los cargo
for script in scripts:
    if script[-3:] == '.py':
        exec(open(str(script)).read())

['/home/egidio/Dropbox/Compucosas/programas py/percentiles.py',
 '/home/egidio/Dropbox/Compucosas/programas py/características_operativas_de_test.py',
 '/home/egidio/Dropbox/Compucosas/programas py/lector_ODF.py',
 '/home/egidio/Dropbox/Compucosas/programas py/como importar',
 '/home/egidio/Dropbox/Compucosas/programas py/tabala_frecuencias.py',
 '/home/egidio/Dropbox/Compucosas/programas py/descripcion_univariada.py',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/características_operativas_de_test.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/tabala_frecuencias.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/lector_ODF.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/percentiles.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/funciones_locales.cpython-36.pyc',
 '/home/egidio/Dropbox/Compucosas/programas py/__pycache__/descripcion_univariada.cpython-36.pyc']

# Recojo la BD

In [8]:
BD = pd.read_pickle('./llamadas_contraspaso.pkl')
BD.head()

,Fecha,Tipo_llamada,Origen_tipo,Destino_tipo,Origen_n,Destino_n,Destino_nSIP,Estado,Duracion,Cobro,Fin,Trapaso
0,2017-02-22 20:54:07,Desde SAMU,SIP,Trunk,204.0,9.904491e+08,NaN,Contestada,12,8,2017-02-22 20:54:19,No
1,2017-02-22 20:54:30,Desde SAMU,SIP,Trunk,204.0,9.992239e+09,NaN,Contestada,20,16,2017-02-22 20:54:50,No
2,2017-02-22 21:21:17,Hacia SAMU,Trunk,SIP,NaN,1.310000e+02,200.0,Contestada,24,14,2017-02-22 21:21:41,No
3,2017-02-22 21:23:01,Hacia SAMU,Trunk,SIP,NaN,1.310000e+02,203.0,Contestada,30,20,2017-02-22 21:23:31,No
4,2017-02-22 21:24:47,Hacia SAMU,Trunk,SIP,NaN,1.310000e+02,200.0,Contestada,63,53,2017-02-22 21:25:50,No


In [15]:
BD.info()
# print('Descrip inicial')
# for entrada in BD.columns:
#     print(entrada)
#     BD[entrada].head()
#     tab_frec(BD[entrada]).head()
#     BD[entrada].describe()
#     print('------------------------------')    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127721 entries, 0 to 1127720
Data columns (total 12 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   Fecha         1127721 non-null  datetime64[ns]
 1   Tipo_llamada  1014913 non-null  category      
 2   Origen_tipo   1127240 non-null  category      
 3   Destino_tipo  1033158 non-null  category      
 4   Origen_n      1075784 non-null  float64       
 5   Destino_n     1067257 non-null  float64       
 6   Destino_nSIP  793090 non-null   float64       
 7   Estado        1127721 non-null  category      
 8   Duracion      1127721 non-null  int64         
 9   Cobro         1127721 non-null  int64         
 10  Fin           1127721 non-null  datetime64[ns]
 11  Trapaso       1127721 non-null  object        
dtypes: category(4), datetime64[ns](2), float64(3), int64(2), object(1)
memory usage: 73.1+ MB


# Rápido análisis de un mes para saber a que nos enfretamos

In [21]:
db1 = BD.loc[BD.Fecha > pd.to_datetime('2020-05-01')]
db1.Fecha.describe()
db1.head()

count                   18270
unique                  18140
top       2020-05-15 20:43:01
freq                        3
first     2020-05-01 00:00:19
last      2020-05-23 23:12:29
Name: Fecha, dtype: object

,Fecha,Tipo_llamada,Origen_tipo,Destino_tipo,Origen_n,Destino_n,Destino_nSIP,Estado,Duracion,Cobro,Fin,Trapaso
1109451,2020-05-01 00:00:19,Hacia SAMU,Trunk,SIP,949799508.0,131.0,204.0,Contestada,567,566,2020-05-01 00:09:46,No
1109452,2020-05-01 00:03:26,Entre SAMU,SIP,SIP,206.0,212.0,212.0,Contestada,112,101,2020-05-01 00:05:18,Si
1109453,2020-05-01 00:04:41,Hacia SAMU,Trunk,SIP,950740068.0,131.0,200.0,Contestada,444,443,2020-05-01 00:12:05,No
1109454,2020-05-01 00:08:59,Entre SAMU,SIP,SIP,204.0,205.0,205.0,Contestada,152,119,2020-05-01 00:11:31,Si
1109455,2020-05-01 00:11:47,Entre SAMU,SIP,SIP,200.0,205.0,205.0,Contestada,140,138,2020-05-01 00:14:07,Si


Descripicón de el periodo de estudio